In [5]:
import pandas as pd
import numpy as np
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from joblib import load
from catboost import CatBoostRegressor

# Load the test data
test_df = pd.read_csv('cleanTest.csv')

# Handle missing values in text columns
test_df[text_columns] = test_df[text_columns].fillna('')

# Combine text columns into one for processing
test_df['combined_text'] = test_df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Load the saved Word2Vec model
word2vec_model = Word2Vec.load('word2vec_model.model')

# Tokenize and preprocess text data for Word2Vec
def preprocess_text(text):
    return simple_preprocess(text, deacc=True)

# Create document vectors by averaging word vectors
def vectorize_text(text):
    tokens = preprocess_text(text)
    vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    if len(vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(vectors, axis=0)

test_df['text_vector'] = test_df['combined_text'].apply(vectorize_text)
word2vec_test_features = np.array(test_df['text_vector'].tolist())

# TF-IDF Vectorization for test data
tfidf_test_vectors = tfidf_vectorizer.transform(test_df['combined_text'])

# Combine Word2Vec and TF-IDF features for test data
combined_test_features = hstack([tfidf_test_vectors, word2vec_test_features])

# Prepare categorical features
X_cat_test = test_df[cat_features]

# Apply the same preprocessor as in training
X_cat_preprocessed_test = preprocessor.transform(X_cat_test)

# Combine categorical and text features
X_test_combined = hstack([X_cat_preprocessed_test, combined_test_features])

# Load the saved CatBoost model
catboost_model = CatBoostRegressor()
catboost_model.load_model('catboost_model.cbm')

# Make predictions on the test data
y_pred_test = catboost_model.predict(X_test_combined)

# Print predictions
print("\nPredicted values for 'degerlendirme_puani':", y_pred_test)


CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:72: Feature 1552 is present in model but not in pool.

NameError: name 'X_train_combined' is not defined